# Uvod u vjerojatnost i statistiku
U ovom bilježniku ćemo se pozabaviti nekim od pojmova koje smo prethodno raspravljali. Mnogi pojmovi iz vjerojatnosti i statistike dobro su zastupljeni u glavnim knjižnicama za obradu podataka u Pythonu, poput `numpy` i `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Slučajne varijable i distribucije
Počnimo s izvlačenjem uzorka od 30 vrijednosti iz jednolikog rasporeda od 0 do 9. Također ćemo izračunati srednju vrijednost i varijancu.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Za vizualno procjenu koliko različitih vrijednosti ima u uzorku, možemo nacrtati **histogram**:


In [ ]:
plt.hist(sample)
plt.show()

## Analiza stvarnih podataka

Srednja vrijednost i varijanca su vrlo važni pri analizi podataka iz stvarnog svijeta. Učitajmo podatke o igračima bejzbola s [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Koristimo paket pod nazivom [**Pandas**](https://pandas.pydata.org/) za analizu podataka. Više ćemo razgovarati o Pandasu i radu s podacima u Pythonu kasnije u ovom tečaju.

Izračunajmo prosječne vrijednosti za dob, visinu i težinu:


In [ ]:
df[['Age','Height','Weight']].mean()

Sada se fokusirajmo na visinu i izračunajmo standardnu devijaciju i varijansu:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Osim srednje vrijednosti, ima smisla pogledati i medijan i kvartile. Oni se mogu vizualizirati pomoću **box plot** grafa:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Također možemo napraviti dijagrame kutija za podskupove našeg skupa podataka, na primjer, grupirane po ulozi igrača.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Napomena**: Ovaj dijagram sugerira da su prosječne visine prvih bazaša veće od visina drugih bazaša. Kasnije ćemo naučiti kako možemo formalnije testirati ovu hipotezu i kako pokazati da su naši podaci statistički značajni za to.  

Dob, visina i težina su sve kontinuirane slučajne varijable. Što mislite kakva im je distribucija? Dobar način da to saznate je da nacrtate histogram vrijednosti: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Normalna raspodjela

Napravimo umjetni uzorak težina koja slijedi normalnu raspodjelu s istim srednjim vrijednostima i varijancom kao i naši stvarni podaci:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Budući da je većina vrijednosti u stvarnom životu normalno raspoređena, ne bismo trebali koristiti jednolik generator slučajnih brojeva za generiranje uzoraka podataka. Evo što se događa ako pokušamo generirati težine s jednolikom raspodjelom (generirano pomoću `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Intervali pouzdanosti

Izračunajmo sada intervale pouzdanosti za težine i visine bejzbol igrača. Upotrijebit ćemo kod [iz ove rasprave na stackoverflowu](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Testiranje hipoteza

Istražimo različite uloge u našem skupu podataka o bejzbol igračima:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Testirajmo hipotezu da su prvašice (First Basemen) viši od drugara na drugoj osnovi (Second Basemen). Najjednostavniji način za to je testirati intervale pouzdanosti:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Vidimo da se intervali ne preklapaju.

Statistički ispravniji način da se dokaže hipoteza je korištenje **Studentovog t-testa**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Dvije vrijednosti koje vraća funkcija `ttest_ind` su:
* p-vrijednost se može smatrati vjerojatnošću da dvije distribucije imaju isti srednju vrijednost. U našem slučaju, vrlo je niska, što znači da postoji snažan dokaz koji podržava da su prva bazaši viši.
* t-vrijednost je međuvrijednost normalizirane razlike srednjih vrijednosti koja se koristi u t-testu, i uspoređuje se s graničnom vrijednošću za zadanu razinu pouzdanosti.


## Simulacija normalne distribucije pomoću središnjeg graničnog teorema

Pseudo-slučajni generator u Pythonu dizajniran je da nam daje uniformnu distribuciju. Ako želimo stvoriti generator za normalnu distribuciju, možemo koristiti središnji granični teorem. Da bismo dobili vrijednost koja je normalno distribuirana, jednostavno ćemo izračunati srednju vrijednost uzorka generiranog uniformno.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Korelacija i Evil Baseball Corp

Korelacija nam omogućuje pronalaženje odnosa između nizova podataka. U našem primjeru, pretvarajmo se da postoji zla bejzbol korporacija koja plaća svoje igrače prema njihovoj visini - što je igrač viši, to više novca dobiva. Pretpostavimo da postoji osnovna plaća od 1000$, i dodatni bonus od 0 do 100$, ovisno o visini. Uzet ćemo prave igrače iz MLB-a i izračunati njihove imaginarne plaće:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Sada ćemo izračunati kovarijancu i korelaciju tih nizova. `np.cov` će nam dati tzv. **matricu kovarijance**, koja je proširenje kovarijance na više varijabli. Element $M_{ij}$ matrice kovarijance $M$ je kovarijanca između ulaznih varijabli $X_i$ i $X_j$, a dijagonalne vrijednosti $M_{ii}$ su varijance $X_{i}$. Slično tome, `np.corrcoef` će nam dati **matricu korelacije**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Korelacija jednaka 1 znači da postoji jaka **linearna veza** između dvije varijable. Linearna veza može se vizualno uočiti tako da se jedna vrijednost prikaže nasuprot druge:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Pogledajmo što se događa ako odnos nije linearan. Pretpostavimo da je naša korporacija odlučila sakriti očitu linearnu ovisnost između visina i plaća, te je u formulu uvela neku nelinearnost, poput `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

U ovom slučaju, korelacija je nešto manja, ali je i dalje prilično visoka. Sada, kako bismo vezu učinili još manje očitom, možda ćemo htjeti dodati dodatnu slučajnost dodavanjem neke slučajne varijable na plaću. Pogledajmo što se događa:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Možete li pogoditi zašto se točkice poravnavaju u vertikalne linije ovako?

Promatrali smo povezanost između umjetno konstruiranog koncepta poput plaće i promatrane varijable *visine*. Pogledajmo također hoće li se dvije promatrane varijable, poput visine i težine, također povezati:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Nažalost, nismo dobili nikakve rezultate - samo neke čudne vrijednosti `nan`. To je zbog činjenice da su neke vrijednosti u našoj seriji nedefinirane, predstavljene kao `nan`, što uzrokuje da i rezultat operacije bude nedefiniran. Promatrajući matricu možemo vidjeti da je `Weight` problematični stupac, jer je izračunata samokorelacija između vrijednosti `Height`.

> Ovaj primjer pokazuje važnost **pripreme podataka** i **čišćenja**. Bez ispravnih podataka ne možemo izračunati ništa.

Koristimo metodu `fillna` za popunjavanje nedostajućih vrijednosti i izračunavanje korelacije:


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Doista postoji korelacija, ali ne tako snažna kao u našem umjetnom primjeru. Zapravo, ako pogledamo dijagram raspršenja jedne vrijednosti u odnosu na drugu, veza bi bila mnogo manje očita:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Zaključak

U ovom smo bilježnicu naučili kako izvoditi osnovne operacije nad podacima za izračun statističkih funkcija. Sada znamo kako koristiti solidan aparat matematike i statistike kako bismo dokazali neke hipoteze i kako izračunati intervale pouzdanosti za proizvoljne varijable s obzirom na uzorak podataka.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Napomena**:
Ovaj dokument preveden je pomoću AI prevoditeljskog servisa [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na njegovom izvornom jeziku treba smatrati ovlaštenim izvorom. Za kritične informacije preporučuje se profesionalni ljudski prijevod. Ne snosimo odgovornost za bilo kakve nesporazume ili pogrešna tumačenja koja proizlaze iz korištenja ovog prijevoda.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
